契約期間２&フル間取りでGOSSに交差検証法を試す。

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold

###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

In [7]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K",'S']]], axis=1)
y_train = rent

#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K",'S']]], axis=1)

category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K','S']

cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    params = {
        'objective':'regression',
        'boosting_type':'goss',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': 1.2667345328336822e-06,
        'max_bin': 405,
        'num_leaves': 118
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001537 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2125
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49712.4	valid_1's rmse: 52813.4
[20]	training's rmse: 35153.7	valid_1's rmse: 40945.6
[30]	training's rmse: 27250.7	valid_1's rmse: 34945.2
[40]	training's rmse: 22563.5	valid_1's rmse: 31659.1
[50]	training's rmse: 19563.3	valid_1's rmse: 29684.6
[60]	training's rmse: 17729.5	valid_1's rmse: 28401.5
[70]	training's rmse: 16460.1	valid_1's rmse: 27590
[80]	training's rmse: 15588.2	valid_1's rmse: 27036.5
[90]	training's rmse: 14881.7	valid_1's rmse: 26543.1
[100]	traini

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001470 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2124
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118478.809819
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50141.5	valid_1's rmse: 51820.2
[20]	training's rmse: 35997.3	valid_1's rmse: 38275.5
[30]	training's rmse: 28619.7	valid_1's rmse: 31518.7
[40]	training's rmse: 24234.4	valid_1's rmse: 27181
[50]	training's rmse: 21447.5	valid_1's rmse: 24580.6
[60]	training's rmse: 19701.8	valid_1's rmse: 23129.9
[70]	training's rmse: 18561.6	valid_1's rmse: 22265.3
[80]	training's rmse: 17603.6	valid_1's rmse: 21694.7
[90]	training's rmse: 16758.2	valid_1's rmse: 21464.5
[100]	traini

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[10]	training's rmse: 50848.1	valid_1's rmse: 49993.6
[20]	training's rmse: 36719.2	valid_1's rmse: 36433.4
[30]	training's rmse: 29318.1	valid_1's rmse: 29109.1
[40]	training's rmse: 24793.6	valid_1's rmse: 25207.7
[50]	training's rmse: 21930	valid_1's rmse: 22737.2
[60]	training's rmse: 20186.2	valid_1's rmse: 21652.2
[70]	training's rmse: 19007.3	valid_1's rmse: 20884
[80]	training's rmse: 18104.3	valid_1's rmse: 20352.7
[90]	training's rmse: 17335.8	valid_1's rmse: 19947.5
[100]	training's rmse: 16714.8	valid_1's rmse: 19729.9
[110]	training's rmse: 16222.5	valid_1's rmse: 19543.1
[120]	training's rmse: 15750.2	valid_1's rmse: 19347.8
[130]	training's rmse: 15397.8	valid_1's rmse: 19247.6
[140]	training's rmse: 15017.9	valid_1's rmse: 19114.1
[150]	training's rmse: 14671.3	valid_1's rmse: 19019.6
[160]	training's rmse: 14404.8	valid_1's rmse: 18966.9
[170]	training's rmse: 14120.7	valid_1's rmse: 18880
[180]	training's rmse: 13840.4	valid_1's rmse: 18828
[190]	training's rmse: 1361

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[10]	training's rmse: 51209	valid_1's rmse: 48697.2
[20]	training's rmse: 36701.6	valid_1's rmse: 36058.3
[30]	training's rmse: 28948.1	valid_1's rmse: 29459.9
[40]	training's rmse: 24335	valid_1's rmse: 25894.1
[50]	training's rmse: 21538.3	valid_1's rmse: 23745.9
[60]	training's rmse: 19696.6	valid_1's rmse: 22383.1
[70]	training's rmse: 18485.9	valid_1's rmse: 21562
[80]	training's rmse: 17631.5	valid_1's rmse: 20975.3
[90]	training's rmse: 16920.5	valid_1's rmse: 20555.5
[100]	training's rmse: 16312.7	valid_1's rmse: 20259.7
[110]	training's rmse: 15841.9	valid_1's rmse: 20044.8
[120]	training's rmse: 15424.6	valid_1's rmse: 19826.5
[130]	training's rmse: 15049.7	valid_1's rmse: 19668.6
[140]	training's rmse: 14672.4	valid_1's rmse: 19487.4
[150]	training's rmse: 14311.9	valid_1's rmse: 19321.5
[160]	training's rmse: 14044.5	valid_1's rmse: 19241.4
[170]	training's rmse: 13701.3	valid_1's rmse: 19096.4
[180]	training's rmse: 13434.3	valid_1's rmse: 19037.5
[190]	training's rmse: 13

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[10]	training's rmse: 50180.5	valid_1's rmse: 53087.8
[20]	training's rmse: 36371.4	valid_1's rmse: 39220.7
[30]	training's rmse: 28682.7	valid_1's rmse: 31627.6
[40]	training's rmse: 24251.4	valid_1's rmse: 26942.5
[50]	training's rmse: 21511.8	valid_1's rmse: 24119.6
[60]	training's rmse: 19944.4	valid_1's rmse: 22533.4
[70]	training's rmse: 18816	valid_1's rmse: 21475.6
[80]	training's rmse: 17886.6	valid_1's rmse: 20754.8
[90]	training's rmse: 17246.2	valid_1's rmse: 20264.5
[100]	training's rmse: 16703.9	valid_1's rmse: 19850.1
[110]	training's rmse: 16178.7	valid_1's rmse: 19553.2
[120]	training's rmse: 15707.8	valid_1's rmse: 19251.7
[130]	training's rmse: 15318.2	valid_1's rmse: 18982.6
[140]	training's rmse: 15004.9	valid_1's rmse: 18778
[150]	training's rmse: 14670.1	valid_1's rmse: 18584.5
[160]	training's rmse: 14385.5	valid_1's rmse: 18354
[170]	training's rmse: 14111.8	valid_1's rmse: 18210.9
[180]	training's rmse: 13883.5	valid_1's rmse: 18094.8
[190]	training's rmse: 13

In [8]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('GOSS_cv_result.csv',index=False)

GOSSをクロスバリデーションでチューニング

In [9]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K",'S']]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)
category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K', 'S']

cv = KFold(n_splits=5, shuffle=True, random_state=0)
rmse_scores = []
X_train['KFold'] = -1
#foldはvalidの対象となる値で０から４
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_train.loc[valid_index,'KFold'] = fold_id
    y_train.loc[valid_index,'KFold'] = fold_id

def objective(trial):
    #foldはvalidの対象となる値で０から４
    for fold in range(5):
        X_tr = X_train[X_train['KFold']!=fold].drop('KFold',axis=1)
        X_val = X_train[X_train['KFold']==fold].drop('KFold',axis=1)
        y_tr = y_train[y_train['KFold']!=fold].drop('KFold',axis=1)
        y_val = y_train[y_train['KFold']==fold].drop('KFold',axis=1)

        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
        params = {
            'objective':'regression',
            'boosting_type':'goss',
            'metrics':'rmse',
            'learning_rate':0.05,
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
            'max_bin': trial.suggest_int('max_bin', 255, 600),
            'num_leaves': trial.suggest_int('num_leaves', 50, 128),
        }

        model = lgb.train(
                            params,
                            lgb_train, 
                            valid_sets=[lgb_train, lgb_eval], 
                            verbose_eval=10, 
                            num_boost_round=3000, 
                            early_stopping_rounds=10,
                            categorical_feature = category_lists
                            )

        y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
        rmse_score = mean_squared_error(y_val, y_pred_val, squared=False)
    loss = np.mean(rmse_score)
    return loss

In [10]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)
study.best_params

[I 2022-09-11 00:10:16,234] A new study created in memory with name: no-name-79777dec-1b56-428e-ad76-ba488dadc443
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001461 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2347
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49657.6	valid_1's rmse: 52813.6
[20]	training's rmse: 35224.3	valid_1's rmse: 40947.7
[30]	training's rmse: 27025.8	valid_1's rmse: 34700.1
[40]	training's rmse: 22360.3	valid_1's rmse: 31303.8
[50]	training's rmse: 19380.3	valid_1's rmse: 29207.6
[60]	training's rmse: 17480.4	valid_1's rmse: 27917.8
[70]	training's rmse: 16241.2	valid_1's rmse: 27091.9
[80]	training's rmse: 15358.7	valid_1's rmse: 26479.9
[90]	training's rmse: 14709.4	valid_1's rmse: 26050.5
[100]	trai

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001283 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2347
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118478.809819
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50762.6	valid_1's rmse: 52740.7
[20]	training's rmse: 36709	valid_1's rmse: 38947.9
[30]	training's rmse: 29215.7	valid_1's rmse: 31588.2
[40]	training's rmse: 24743.7	valid_1's rmse: 27266.5
[50]	training's rmse: 21977.1	valid_1's rmse: 24631.6
[60]	training's rmse: 20245.3	valid_1's rmse: 23048.1
[70]	training's rmse: 18847.9	valid_1's rmse: 22400
[80]	training's rmse: 17941.6	valid_1's rmse: 21793.5
[90]	training's rmse: 17208.8	valid_1's rmse: 21713.8
[100]	training

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

...

[10]	training's rmse: 51042.8	valid_1's rmse: 48830.2
[20]	training's rmse: 36452.6	valid_1's rmse: 36129.2
[30]	training's rmse: 28725.4	valid_1's rmse: 29386.3
[40]	training's rmse: 24033.8	valid_1's rmse: 25742.4
[50]	training's rmse: 21229.8	valid_1's rmse: 23596.2
[60]	training's rmse: 19595.5	valid_1's rmse: 22333.7
[70]	training's rmse: 18379.9	valid_1's rmse: 21553.1
[80]	training's rmse: 17521.5	valid_1's rmse: 21055
[90]	training's rmse: 16859.7	valid_1's rmse: 20713.7
[100]	training's rmse: 16362.1	valid_1's rmse: 20457.8
[110]	training's rmse: 15852.3	valid_1's rmse: 20325.5
[120]	training's rmse: 15439	valid_1's rmse: 20196.9
[130]	training's rmse: 15078.2	valid_1's rmse: 20039.2
[140]	training's rmse: 14710	valid_1's rmse: 19893
[150]	training's rmse: 14387.6	valid_1's rmse: 19722.3
[160]	training's rmse: 14115.8	valid_1's rmse: 19631.9
[170]	training's rmse: 13857.9	valid_1's rmse: 19546.1
[180]	training's rmse: 13634.2	valid_1's rmse: 19464.1
[190]	training's rmse: 1341

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001667 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2461
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118197.706188
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50396.1	valid_1's rmse: 53168
[20]	training's rmse: 36591.4	valid_1's rmse: 39419.3
[30]	training's rmse: 29124.5	valid_1's rmse: 31835.1
[40]	training's rmse: 24662.6	valid_1's rmse: 27331.1
[50]	training's rmse: 21968.8	valid_1's rmse: 24431.2
[60]	training's rmse: 20222.2	valid_1's rmse: 22683.2
[70]	training's rmse: 19068.9	valid_1's rmse: 21583.6
[80]	training's rmse: 18199.5	valid_1's rmse: 20839.8
[90]	training's rmse: 17543.9	valid_1's rmse: 20316.1
[100]	traini

[I 2022-09-11 00:20:18,964] Trial 39 finished with value: 16248.958091395272 and parameters: {'reg_lambda': 5.543941231681784e-06, 'max_bin': 560, 'num_leaves': 104}. Best is trial 36 with value: 15783.79718784424.


{'reg_lambda': 4.669226029558975e-07, 'max_bin': 584, 'num_leaves': 85}

In [14]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K",'S']]], axis=1)
y_train = rent

#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K",'S']]], axis=1)

category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K','S']

cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    params = {
        'objective':'regression',
        'boosting_type':'goss',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': 4.669226029558975e-07,
        'max_bin': 584,
        'num_leaves': 85,
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001786 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2511
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49876.2	valid_1's rmse: 53024.8
[20]	training's rmse: 35307.7	valid_1's rmse: 41412.7
[30]	training's rmse: 27306.5	valid_1's rmse: 35496.8
[40]	training's rmse: 22571.2	valid_1's rmse: 32195.9
[50]	training's rmse: 19575.3	valid_1's rmse: 30099.9
[60]	training's rmse: 17719	valid_1's rmse: 28847.6
[70]	training's rmse: 16476.5	valid_1's rmse: 27952
[80]	training's rmse: 15633.6	valid_1's rmse: 27344.4
[90]	training's rmse: 14954.1	valid_1's rmse: 26862.1
[100]	training

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000881 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2511
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118478.809819
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50637.1	valid_1's rmse: 52141.1
[20]	training's rmse: 36517.2	valid_1's rmse: 38397.3
[30]	training's rmse: 28867	valid_1's rmse: 30910.1
[40]	training's rmse: 24463.8	valid_1's rmse: 26862.3
[50]	training's rmse: 21656.2	valid_1's rmse: 24128.6
[60]	training's rmse: 19878.1	valid_1's rmse: 22743.5
[70]	training's rmse: 18736.3	valid_1's rmse: 21818.8
[80]	training's rmse: 17915.8	valid_1's rmse: 21248.6
[90]	training's rmse: 17233.1	valid_1's rmse: 20863.6
[100]	traini

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[30]	training's rmse: 29116.1	valid_1's rmse: 28820.8
[40]	training's rmse: 24771.4	valid_1's rmse: 25108.8
[50]	training's rmse: 21985.8	valid_1's rmse: 22925.4
[60]	training's rmse: 20210.6	valid_1's rmse: 21649.9
[70]	training's rmse: 19039.7	valid_1's rmse: 20918.8
[80]	training's rmse: 18206.5	valid_1's rmse: 20526.1
[90]	training's rmse: 17509.4	valid_1's rmse: 20226.1
[100]	training's rmse: 16902.2	valid_1's rmse: 19983
[110]	training's rmse: 16376.2	valid_1's rmse: 19834.1
[120]	training's rmse: 15998.8	valid_1's rmse: 19712.5
[130]	training's rmse: 15631.8	valid_1's rmse: 19644.4
[140]	training's rmse: 15256.5	valid_1's rmse: 19482.1
[150]	training's rmse: 14908.2	valid_1's rmse: 19423.1
[160]	training's rmse: 14629.5	valid_1's rmse: 19383
[170]	training's rmse: 14344.6	valid_1's rmse: 19346.8
[180]	training's rmse: 14067.3	valid_1's rmse: 19302.5
[190]	training's rmse: 13830.8	valid_1's rmse: 19276.1
[200]	training's rmse: 13633.5	valid_1's rmse: 19206.9
[210]	training's rmse

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001248 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2513
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118323.873650
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 51147.6	valid_1's rmse: 48761.9
[20]	training's rmse: 36619.6	valid_1's rmse: 36148.2
[30]	training's rmse: 28793.7	valid_1's rmse: 29575.4
[40]	training's rmse: 24141.7	valid_1's rmse: 25824.3
[50]	training's rmse: 21337.8	valid_1's rmse: 23570.8
[60]	training's rmse: 19673.6	valid_1's rmse: 22233.3
[70]	training's rmse: 18402.2	valid_1's rmse: 21450.3
[80]	training's rmse: 17557.2	valid_1's rmse: 21054.6
[90]	training's rmse: 16881.7	valid_1's rmse: 20740.8
[100]	trai

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' ar

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000853 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2509
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118197.706188
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50545.8	valid_1's rmse: 53286.4
[20]	training's rmse: 36855	valid_1's rmse: 39439.6
[30]	training's rmse: 29335.2	valid_1's rmse: 31791.1
[40]	training's rmse: 24928.2	valid_1's rmse: 27128.2
[50]	training's rmse: 22146.9	valid_1's rmse: 24183.5
[60]	training's rmse: 20325.5	valid_1's rmse: 22328.4
[70]	training's rmse: 19188.2	valid_1's rmse: 21192.1
[80]	training's rmse: 18283.5	valid_1's rmse: 20549.2
[90]	training's rmse: 17638	valid_1's rmse: 20094.8
[100]	training

In [15]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('GOSS_cv_result.csv',index=False)